In [ ]:
import os
import sys
import random
import itertools
import importlib
import pandas as pd
import nibabel as nib
from datetime import datetime

sys.path.append('/host/verges/tank/data/daniel/00_commonUtils/00_code/genUtils/')
#print("Contents:", os.listdir('/host/verges/tank/data/daniel/00_commonUtils/00_code/genUtils/'))
import visUtils
import gen
import stitchSurfs as stitch
import bids_naming as names
import dataChecks as check

NameError: name 'dataclass' is not defined

In [ ]:
# parameters

test = True
test_n = 2
verbose = True

demographics_pth = '/host/verges/tank/data/daniel/00_commonUtils/01_demographics/02_combined/demographics_23Jan2026-161349.csv'

project = {
    'dir_root': '/host/verges/tank/data/daniel/04_inVivoHistology/',
    'dir_data': 'data/',
    'dir_out': 'outputs/'
}

features = ["T1map"]

pni_resolution = (0.5, 0.5, 0.5)  # target resolution for PNI 7T scans

mp_surfaces = {
    'surf': ['fsLR-32k'],
    'lbl': ['pial', 'white'] # required input labels
}
hu_surfaces = {
    'surf': ['den-0p5mm'],
    'lbl': ['inner', 'outer']
}

ctx_hipp_lblCorrsespondence = {
    'pial': 'outer',
    'white': 'inner'
}

PNI = {
    'dir_root': '/data/mica3/BIDS_PNI/',
    'dir_deriv': 'derivatives/',
    'dir_mp': 'micapipe_v0.2.0/',
    'dir_hu': 'hippunfold_v1.3.0/hippunfold/', # update to v2?
}

MICS = {
    'dir_root': '/data/mica3/BIDS_MICs/',
    'dir_deriv': 'derivatives/',
    'dir_mp': 'micapipe_v0.2.0/',
    'dir_hu': 'hippunfold_v1.3.0/hippunfold/', # update to v2?
}

varsOfInterest = ['UID', 'MICS_ID', 'PNI_ID', 'study', 'SES', 'Date', 'sex', 'age', 'grp','grp_detailed']


In [6]:
# input: demographics datasheet
demo = pd.read_csv(demographics_pth)
print("Shape of demographics:", demo.shape)
print("Columns in demographics:", demo.columns.tolist())
demo.head()

Shape of demographics: (871, 48)
Columns in demographics: ['UID', 'MICS_ID', 'PNI_ID', 'study', 'SES', 'Date', 'gender', 'sex', 'employment', 'language', 'dob', 'education', 'ethnicity', 'lastSeizure', 'handedness', 'Scan_Date (D.M.Y)', 'WeightApprox', 'Handed', 'Employ', 'YoE', 'AssignedSex', 'HeightApprox', 'ASMs  on admission (name, doses (mg per day)', 'Epilepsy diagnosis based on ILAE', 'FDG.PET', 'Duration of admission', 'Surgical resection date and site', 'Invasive explorations (Y/N)', 'Dx at EMU discharge ', 'Baseline MRI (year,results)', 'Epilepsy classification:Focal,Generalized', 'Lateralization of epileptogenic focus', 'Engel classification (seizure outcomes at the 6 month )', 'Epileptogenic focus based on EMU information', 'Drug resistant epilepsy at time of EMU admission', 'Histopatholgy', 'Previous ASMs (name and doses (mg/d)) if applicable prior the current EMU admission', '# of ASM on admission', 'Risk factors for epilepsy', '# of ASMs prior current EMU admission', 'IL

,UID,MICS_ID,PNI_ID,study,SES,Date,gender,sex,employment,language,...,Risk factors for epilepsy,# of ASMs prior current EMU admission,ILAE outcome after surgical resection by 1 yr,Engel classification (seizure outcomes after 1 year from surgical resection),Neuromodulation devices,EMU admission date(dd-mm-yy),# of surgical resection/thermocoagulatin,age,grp,grp_detailed
0,UID0001,NaN,Pilot007,7T,00,11.03.2022,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CTRL,CTRL
1,UID0002,NaN,Pilot011,7T,05,28.03.2024,NaN,NaN,Full time student,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CTRL,CTRL
2,UID0003,NaN,Pilot012,7T,05,11.04.2024,NaN,NaN,Full time student,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CTRL,CTRL
3,UID0004,HC129,Pilot013,7T,05,18.04.2024,NaN,F,Full time student,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.728953,CTRL,CTRL
4,UID0004,HC129,Pilot013,3T,01,09.07.2024,NaN,F,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.953457,CTRL,CTRL


In [ ]:
# 7T only
importlib.reload(gen)
importlib.reload(names)
importlib.reload(check)
qc_cols = []

sT_pt = demo[demo['PNI_ID'].isna()==False][varsOfInterest]
sT_sessions = sT_pt[sT_pt['study'] == '7T']
print(f'Number of 7T sessions: {len(sT_sessions)}')
#print(sT_sessions.head())

print("CHECKING DATA...")
print(f"\t VOLUMES existence...")
demo_volCheck, cols_volChk = check.vol_check(PNI, sT_sessions, features, verbose=verbose)
qc_cols += cols_volChk

print(f"\t RESOLUTION match {pni_resolution}...")
demo_res, res_dictList, cols_resChk = check.resolution_check(demo = sT_sessions, study = PNI, res_trgt = pni_resolution, epsilon=0.001, verbose = verbose)
qc_cols += cols_resChk

print(f"\t SURFACES present...")
demo_volSurf_check, cols_procChk = check.proc_check(study = PNI, demo = demo_volCheck, mp_surfaces = mp_surfaces, hu_surfaces = hu_surfaces, verbose=verbose)
qc_cols += cols_procChk

# check QC of surfaces and volumes
# TODO.

# save
save_pth = os.path.join(project['dir_root'], project['dir_out'], f'demo_QCcols_{gen.fmt_now()}.csv')
demo_volSurf_check.to_csv(save_pth, index=False)
print(f"[main] Saved demographics with QC columns to: {save_pth}")


Number of 7T sessions: 210
CHECKING DATA...
	 VOLUMES existence...
	Adding volume check column: hasVol_T1map
		Missing volume | Pilot007-00 can't find: /data/mica3/BIDS_PNI/derivatives/micapipe_v0.2.0/sub-Pilot007/ses-00/maps/sub-Pilot007_ses-00_space-nativepro_map-T1map.nii.gz


/host/verges/tank/data/daniel/04_inVivoHistology/code/dataChecks.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demo[col_name] = True


		Missing volume | PNC008-01 can't find: /data/mica3/BIDS_PNI/derivatives/micapipe_v0.2.0/sub-PNC008/ses-01/maps/sub-PNC008_ses-01_space-nativepro_map-T1map.nii.gz
		Missing volume | PNC032-a3 can't find: /data/mica3/BIDS_PNI/derivatives/micapipe_v0.2.0/sub-PNC032/ses-a3/maps/sub-PNC032_ses-a3_space-nativepro_map-T1map.nii.gz
		Missing volume | PNE002-a1 can't find: /data/mica3/BIDS_PNI/derivatives/micapipe_v0.2.0/sub-PNE002/ses-a1/maps/sub-PNE002_ses-a1_space-nativepro_map-T1map.nii.gz
		Missing volume | PNE007-a3 can't find: /data/mica3/BIDS_PNI/derivatives/micapipe_v0.2.0/sub-PNE007/ses-a3/maps/sub-PNE007_ses-a3_space-nativepro_map-T1map.nii.gz
		Missing volume | PNE032-a1 can't find: /data/mica3/BIDS_PNI/derivatives/micapipe_v0.2.0/sub-PNE032/ses-a1/maps/sub-PNE032_ses-a1_space-nativepro_map-T1map.nii.gz
		Missing volume | PNE050-a1 can't find: /data/mica3/BIDS_PNI/derivatives/micapipe_v0.2.0/sub-PNE050/ses-a1/maps/sub-PNE050_ses-a1_space-nativepro_map-T1map.nii.gz
		Missing volume

/host/verges/tank/data/daniel/04_inVivoHistology/code/dataChecks.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demo[col_name] = True
/host/verges/tank/data/daniel/04_inVivoHistology/code/dataChecks.py:91: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  demo.loc[idx, col_name] = np.nan


	File does not exist	| PNC008-01: /data/mica3/BIDS_PNI/derivatives/micapipe_v0.2.0/sub-PNC008/ses-01/maps/sub-PNC008_ses-01_space-nativepro_map-T1map.nii.gz
	File does not exist	| PNC032-a3: /data/mica3/BIDS_PNI/derivatives/micapipe_v0.2.0/sub-PNC032/ses-a3/maps/sub-PNC032_ses-a3_space-nativepro_map-T1map.nii.gz
	File does not exist	| PNE002-a1: /data/mica3/BIDS_PNI/derivatives/micapipe_v0.2.0/sub-PNE002/ses-a1/maps/sub-PNE002_ses-a1_space-nativepro_map-T1map.nii.gz
	File does not exist	| PNE007-a3: /data/mica3/BIDS_PNI/derivatives/micapipe_v0.2.0/sub-PNE007/ses-a3/maps/sub-PNE007_ses-a3_space-nativepro_map-T1map.nii.gz
	File does not exist	| PNE032-a1: /data/mica3/BIDS_PNI/derivatives/micapipe_v0.2.0/sub-PNE032/ses-a1/maps/sub-PNE032_ses-a1_space-nativepro_map-T1map.nii.gz
	File does not exist	| PNE050-a1: /data/mica3/BIDS_PNI/derivatives/micapipe_v0.2.0/sub-PNE050/ses-a1/maps/sub-PNE050_ses-a1_space-nativepro_map-T1map.nii.gz
	File does not exist	| PNE055-a1: /data/mica3/BIDS_PNI/der

/host/verges/tank/data/daniel/04_inVivoHistology/code/dataChecks.py:138: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demo[col] = False
/host/verges/tank/data/daniel/04_inVivoHistology/code/dataChecks.py:138: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demo[col] = False
/host/verges/tank/data/daniel/04_inVivoHistology/code/dataChecks.py:142: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

	Neither L nor R surface file exists	| ('/data/mica3/BIDS_PNI/derivatives/micapipe_v0.2.0/sub-PNC008/ses-01/surf/sub-PNC008_ses-01_hemi-L_space-nativepro_surf-fsLR-5k_label-pial.surf.gii', '/data/mica3/BIDS_PNI/derivatives/micapipe_v0.2.0/sub-PNC008/ses-01/surf/sub-PNC008_ses-01_hemi-R_space-nativepro_surf-fsLR-5k_label-pial.surf.gii')
	Neither L nor R surface file exists	| ('/data/mica3/BIDS_PNI/derivatives/micapipe_v0.2.0/sub-PNC008/ses-01/surf/sub-PNC008_ses-01_hemi-L_space-nativepro_surf-fsLR-5k_label-white.surf.gii', '/data/mica3/BIDS_PNI/derivatives/micapipe_v0.2.0/sub-PNC008/ses-01/surf/sub-PNC008_ses-01_hemi-R_space-nativepro_surf-fsLR-5k_label-white.surf.gii')
	Neither L nor R surface file exists	| ('/data/mica3/BIDS_PNI/derivatives/hippunfold_v1.3.0/hippunfold/sub-PNC008/ses-01/surf/sub-PNC008_ses-01_hemi-L_space-T1w_den-0p5mm_label-hipp_inner.surf.gii', '/data/mica3/BIDS_PNI/derivatives/hippunfold_v1.3.0/hippunfold/sub-PNC008/ses-01/surf/sub-PNC008_ses-01_hemi-R_space-T1w_de

AttributeError: module 'gen' has no attribute 'fmt_now'

In [ ]:
reimport = False
if demo_volSurf_check not in locals() or reimport:
    demo_pth = "/host/verges/tank/data/daniel/04_inVivoHistology/outputs/demo_QCcols_28Jan2026-1104.csv"
    demo_volSurf_check = pd.read_csv(demo_pth)
    #qc_cols 3 could save this as an object and read in as well

# filter out bad QC
qc_cols_present = [col for col in qc_cols if col in demo_volSurf_check.columns]
demo_qc, rm_rows = check.filter_qcCols(demo_volSurf_check, qc_cols)

if test: # take
    # take random sample of test_n subjects
    demo_qc = demo_qc.sample(n=test_n, random_state=42)
    print(f"TEST MODE: {test_n} rows kept:")
    for idx, row in demo_qc.iterrows():
        print(f"\t{row['UID']}@{row['study']}: {row['MICS_ID']}-{row['PNI_ID']}-{row['SES']}")

[filter_qcCols] 19 rows (17 unique participants) removed when filtering QC cols: ['hasVol_T1map', 'properRes', 'mp_proc__pial_fsLR-5k', 'mp_proc__white_fsLR-5k', 'hu_proc__inner_den-0p5mm', 'hu_proc__outer_den-0p5mm']
TEST MODE: 2 rows kept:
	UID0085@7T: nan-PNF004-a1
	UID0032@7T: HC062-PNC019-a1


In [ ]:
importlib.reload(visUtils)

demo_qc.head()
for pt in demo_qc.itertuples():
    uid = pt.UID
    study = pt.study
    ses = pt.SES
    mics_id = pt.MICS_ID
    pni_id = pt.PNI_ID
    if study == '7T':
        id = pni_id
        study_dict = PNI
    else:
        id = mics_id
        study_dict = MICS

    mp_root = study_dict['dir_root'] + study_dict['dir_deriv'] + study_dict['dir_mp']
    hu_root = study_dict['dir_root'] + study_dict['dir_deriv'] + study_dict['dir_hu']

    # get paths
    

    surf_pths = []
    for lbl, surf in itertools.product(mp_surfaces['lbl'], mp_surfaces['surf']):
        surf_pth = names.get_surf_pth(root = mp_root, sub = id, ses = ses, lbl=lbl, surf=surf, verbose = False)
        surf_pths += surf_pth
    for lbl, surf in itertools.product(hu_surfaces['lbl'], hu_surfaces['surf']):
        surf_pth = names.get_surf_pth(root = hu_root, sub = id, ses = ses, lbl=lbl, surf=surf, verbose = False)
        surf_pths += surf_pth
    


    # generate visualization command
    vol_pth = names.get_volPath(study = study_dict, id = id, ses = ses, volName = 'T1map')[0]
    print(visUtils.getCMD_freeView(surf_pths=surf_pths, vol_pth=vol_pth))


freeview -f /data/mica3/BIDS_PNI/derivatives/micapipe_v0.2.0/sub-PNF004/ses-a1/surf/sub-PNF004_ses-a1_hemi-L_space-nativepro_surf-fsLR-5k_label-pial.surf.gii:edgecolor=red -f /data/mica3/BIDS_PNI/derivatives/micapipe_v0.2.0/sub-PNF004/ses-a1/surf/sub-PNF004_ses-a1_hemi-R_space-nativepro_surf-fsLR-5k_label-pial.surf.gii:edgecolor=red -f /data/mica3/BIDS_PNI/derivatives/micapipe_v0.2.0/sub-PNF004/ses-a1/surf/sub-PNF004_ses-a1_hemi-L_space-nativepro_surf-fsLR-5k_label-white.surf.gii:edgecolor=yellow -f /data/mica3/BIDS_PNI/derivatives/micapipe_v0.2.0/sub-PNF004/ses-a1/surf/sub-PNF004_ses-a1_hemi-R_space-nativepro_surf-fsLR-5k_label-white.surf.gii:edgecolor=yellow -f /data/mica3/BIDS_PNI/derivatives/hippunfold_v1.3.0/hippunfold/sub-PNF004/ses-a1/surf/sub-PNF004_ses-a1_hemi-L_space-T1w_den-0p5mm_label-hipp_inner.surf.gii:edgecolor=orange -f /data/mica3/BIDS_PNI/derivatives/hippunfold_v1.3.0/hippunfold/sub-PNF004/ses-a1/surf/sub-PNF004_ses-a1_hemi-R_space-T1w_den-0p5mm_label-hipp_inner.surf.

In [ ]:
# stitch cortical and hippocampal surfaces together. NOTE. Jordan code

for pt in demo_qc.itertuples():
    uid = pt.UID
    study = pt.study
    ses = pt.SES
    mics_id = pt.MICS_ID
    pni_id = pt.PNI_ID
    if study == '7T':
        id = pni_id
        study_dict = PNI
        prj_out_study_pth = os.path.join(project['dir_root'], project['dir_data'], "PNI")
    elif study == 'MICs':
        id = mics_id
        study_dict = MICS
        prj_out_study_pth = os.path.join(project['dir_root'], project['dir_data'], "MICs")
    else:
        print(f"[main] WARNING: study {study} not recognized. Skipping subject {uid}...")
        continue

    mp_root = study_dict['dir_root'] + study_dict['dir_deriv'] + study_dict['dir_mp']
    hu_root = study_dict['dir_root'] + study_dict['dir_deriv'] + study_dict['dir_hu']
    
    out_dir = os.path.join(prj_out_study_pth, f'sub-{id}_ses-{ses}/')
    os.mkdir(out_dir)
    
    # iterate through corresponding surfaces
    for mp_surf_lbl, hu_surf_lbl in ctx_hipp_lblCorrsespondence.items():
        print(f"[main] Stitching {mp_surf_lbl} ({mp_root}) with {hu_surf_lbl} ({hu_root}) for sub-{id}_ses-{ses}...")
        
        
        #out_name = f'sub-{id}_ses-{ses}_stitch-{mp_surf["surf"]}_{hu_surf["surf"]}'
        #pth_stitched = stitch.stitchSurfs(mp_surf, hu_surf, out_dir)


#ctx, hipp, out_dir:str, out_name:str, template:str = "overlap_stitch_template.pkl"

NameError: name 'ctx_hipp_lblCorrsespondence' is not defined

In [ ]:


# following steps should be able to be performed on data from any source (PNI, MICs, AHEAD, BigBrain, etc.)

# save surfaces generated at different depths to study directory
# stitch cortical and hippocampal surfaces together. NOTE. Jordan code
# remove vertices not in mesial temporal lobe (choose cut-off limit as per Paquola 2020)
# assign each vertex values along anterior-posterior, and allo/neo-cortical axes
# project features onto surfaces


In [ ]:
# analyses